In [1]:
import torch
from torch import Tensor
from torch_geometric.logging import init_wandb, log
from torch_geometric.datasets import Planetoid
from utils import train, test, edgeindex2adj
from models import GCN, GAT
citeseer = Planetoid(root='.', name='Citeseer')
cora = Planetoid(root='.', name='Cora')

In [2]:
# dataset = citeseer
# model = GCN(dataset.num_features, 24, dataset.num_classes)

# dataset = cora
# model = GCN(dataset.num_features, 8, dataset.num_classes)

# dataset = citeseer
# model = GAT(dataset.num_features, 8, dataset.num_classes, heads=4)

dataset = cora
ei = dataset[0].edge_index
adj = edgeindex2adj(ei, dataset.x.shape[0])
model = GAT(dataset.num_features, 8, dataset.num_classes, heads=4, adj=adj)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

In [3]:
data = dataset[0]

best_val_acc = final_test_acc = 0
for epoch in range(1, 1000):
    loss = train(model, data, optimizer, loss='smooth_label')
    train_acc, val_acc, tmp_test_acc = test(model, data)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
print(f'Best Val Acc: {best_val_acc:.4f}', f'Test Acc: {test_acc:.4f}')

Epoch: 001, Loss: 2.2466883659362793, Train: 0.3500, Val: 0.2700, Test: 0.2780
Epoch: 002, Loss: 2.14047908782959, Train: 0.3571, Val: 0.2940, Test: 0.2660
Epoch: 003, Loss: 2.0529446601867676, Train: 0.4143, Val: 0.3200, Test: 0.3020
Epoch: 004, Loss: 1.954749345779419, Train: 0.4786, Val: 0.3880, Test: 0.3550
Epoch: 005, Loss: 1.9607235193252563, Train: 0.5786, Val: 0.4240, Test: 0.4040
Epoch: 006, Loss: 1.9328062534332275, Train: 0.7000, Val: 0.5100, Test: 0.4780
Epoch: 007, Loss: 1.8627859354019165, Train: 0.7857, Val: 0.5740, Test: 0.5520
Epoch: 008, Loss: 1.8405307531356812, Train: 0.8643, Val: 0.6400, Test: 0.6380
Epoch: 009, Loss: 1.8063565492630005, Train: 0.9000, Val: 0.6880, Test: 0.6820
Epoch: 010, Loss: 1.7706124782562256, Train: 0.9429, Val: 0.7240, Test: 0.7350
Epoch: 011, Loss: 1.784828543663025, Train: 0.9500, Val: 0.7320, Test: 0.7470
Epoch: 012, Loss: 1.7852877378463745, Train: 0.9571, Val: 0.7360, Test: 0.7530
Epoch: 013, Loss: 1.782850980758667, Train: 0.9786, Val: